# Testing & Debugging (v0.14.0)

This notebook demonstrates the testing and debugging utilities in SurrealDB-ORM: **fixtures** for declarative test data, **model factories** for random data generation, and **query logging** for profiling and debugging.

## Use Case: Writing Tests for Your ORM Application

Good tests need:
1. **Repeatable test data** -- fixtures that set up and tear down cleanly
2. **Random data generation** -- factories for property-based testing and load testing
3. **Query visibility** -- see exactly what SQL hits the database and how long it takes

These tools make your test suite faster to write, more reliable, and easier to debug.

## Prerequisites

- SurrealDB running locally (`docker run --rm -p 8000:8000 surrealdb/surrealdb:latest start --user root --pass root`)
- Project dependencies installed (`uv sync`)
- A `.env` file in the project root (optional, falls back to defaults)

In [ ]:
# Setup: add project root to path and configure SurrealDB connection
import os, sys
project_root = os.path.abspath(os.path.join(os.getcwd(), ".."))
sys.path.append(project_root)
from dotenv import load_dotenv
load_dotenv()

from src.surreal_orm import SurrealDBConnectionManager

SurrealDBConnectionManager.set_connection(
    os.getenv("SURREALDB_URL", "ws://localhost:8000"),
    os.getenv("SURREALDB_USER", "root"),
    os.getenv("SURREALDB_PASS", "root"),
    os.getenv("SURREALDB_NAMESPACE", "ns"),
    os.getenv("SURREALDB_DATABASE", "db"),
)

In [ ]:
# Define a model we'll use throughout this notebook
from src.surreal_orm import BaseSurrealModel, SurrealConfigDict

class TestUser(BaseSurrealModel):
    model_config = SurrealConfigDict(table_name="test_user")

    id: str | None = None
    name: str
    role: str = "user"
    score: int = 0

## 1. Test Fixtures -- Declarative Test Data

Fixtures let you declare test data as class attributes. When loaded, they save all records to the database. When the context manager exits, they clean up automatically.

**When to use fixtures:**
- Integration tests that need known data in the database
- Setup/teardown patterns where cleanup must be guaranteed
- Sharing test data definitions across multiple test functions

In [ ]:
# Define fixtures as a class with model instances as attributes
from src.surreal_orm.testing import SurrealFixture, fixture

@fixture
class UserFixtures(SurrealFixture):
    alice = TestUser(name="Alice", role="admin", score=100)
    bob = TestUser(name="Bob", role="player", score=50)

print("Fixtures defined: alice (admin), bob (player)")
print("They haven't been saved to the database yet.")

In [ ]:
# Load fixtures into the database using async context manager
# On enter: saves all fixture records
# On exit: deletes all fixture records (guaranteed cleanup)
async with UserFixtures.load() as fixtures:
    print(f"Alice ID: {fixtures.alice.get_id()}")
    print(f"Bob ID: {fixtures.bob.get_id()}")

    # Query the database -- fixtures are real records
    users = await TestUser.objects().all()
    print(f"\nTotal users in DB: {len(users)}")
    for u in users:
        print(f"  - {u.name} ({u.role}), score: {u.score}")

# After the context manager exits, fixtures are cleaned up
remaining = await TestUser.objects().all()
print(f"\nAfter cleanup: {len(remaining)} users in DB")

## 2. Model Factories -- Random Data Generation

Factories generate model instances with random but realistic data. Use `build()` for in-memory instances (unit tests) or `create()` to save to the database (integration tests).

**When to use factories:**
- Property-based testing with random inputs
- Load testing with large datasets
- When you need many records but don't care about specific values

In [ ]:
# Define a factory with Faker providers for random data
from src.surreal_orm.testing import ModelFactory, Faker

class UserFactory(ModelFactory):
    class Meta:
        model = TestUser

    name = Faker("name")
    role = Faker("choice", items=["admin", "player", "viewer"])
    score = Faker("random_int", min=0, max=1000)

print("UserFactory defined with random name, role, and score.")

In [ ]:
# build() creates instances WITHOUT saving to the database
# Perfect for unit tests where you don't need DB state
user = UserFactory.build()
print(f"Built (not saved): {user.name} ({user.role}), score: {user.score}")
print(f"ID is None (not persisted): {user.id}")

# build_batch() creates multiple instances at once
users = UserFactory.build_batch(5)
print(f"\nBuilt {len(users)} users:")
for u in users:
    print(f"  - {u.name}: {u.role}, score: {u.score}")

In [ ]:
# Override specific fields while keeping others random
# Useful for testing specific scenarios
admin = UserFactory.build(role="admin", score=999)
print(f"Override example: {admin.name} is {admin.role} with score {admin.score}")

## 3. Factory -- create() and create_batch()

Use `create()` and `create_batch()` when you need the records to actually exist in the database (integration tests).

In [ ]:
# create() builds AND saves to the database in one call
user = await UserFactory.create(role="admin")
print(f"Created admin: {user.name}, ID: {user.get_id()}, score: {user.score}")

# create_batch() creates and saves multiple records
players = await UserFactory.create_batch(3, role="player")
print(f"\nCreated {len(players)} players:")
for p in players:
    print(f"  - {p.name} (ID: {p.get_id()}), score: {p.score}")

In [ ]:
# Verify the records exist in the database
all_users = await TestUser.objects().all()
print(f"Total users in DB: {len(all_users)}")

# Cleanup the factory-created records
for u in [user, *players]:
    await u.delete()
print("Factory records cleaned up.")

## 4. Faker Providers Reference

The `Faker` class provides various data generators. Each call to `.generate()` produces a new random value.

In [ ]:
# Demonstrate all available Faker providers
from src.surreal_orm.testing import Faker

providers = [
    ("name", {}),
    ("first_name", {}),
    ("last_name", {}),
    ("email", {}),
    ("random_int", {"min": 1, "max": 100}),
    ("random_float", {"min": 0.0, "max": 1.0}),
    ("text", {"max_length": 50}),
    ("sentence", {}),
    ("word", {}),
    ("uuid", {}),
    ("boolean", {}),
    ("date", {}),
    ("datetime", {}),
    ("choice", {"items": ["a", "b", "c"]}),
]

print("Available Faker providers:")
for name, kwargs in providers:
    value = Faker(name, **kwargs).generate()
    print(f"  Faker({name!r}): {value!r}")

## 5. QueryLogger -- Debug and Profile Queries

The `QueryLogger` captures all SurrealQL queries executed within its context, along with timing information. Use it to:

- Debug unexpected query patterns (N+1 queries, missing filters)
- Profile query performance (find slow queries)
- Verify that the ORM generates the SQL you expect

The logger uses `contextvars`, so it's async-safe and only captures queries from the current async context.

In [ ]:
# Create some test data for query logging
test_users = await UserFactory.create_batch(3)
print(f"Created {len(test_users)} test users for profiling.")

In [ ]:
# QueryLogger captures all queries within the async with block
from src.surreal_orm.debug import QueryLogger

async with QueryLogger() as logger:
    # These queries will be captured
    users = await TestUser.objects().all()
    user = UserFactory.build(name="LoggerTest")
    await user.save()
    await user.delete()

# After the block, inspect the captured queries
print(f"Total: {logger.total_queries} queries in {logger.total_ms:.1f}ms")
print("\nCaptured queries:")
for q in logger.queries:
    print(f"  [{q.duration_ms:.1f}ms] {q.sql}")

In [ ]:
# Use QueryLogger to detect N+1 query problems
# If you see many similar queries in a loop, you likely need prefetch_related()
async with QueryLogger() as logger:
    # This is the "bad" pattern -- one query per user
    users = await TestUser.objects().all()
    for u in users:
        # Imagine fetching related records one by one here
        _ = await TestUser.objects().get(u.get_id())

print(f"N+1 detection: {logger.total_queries} queries for {len(users)} users")
print("If this number grows linearly with record count, use prefetch_related()!")

## QueryLogger Properties

| Property | Type | Description |
|---|---|---|
| `logger.queries` | `list[QueryLog]` | All captured query objects |
| `logger.total_queries` | `int` | Number of queries captured |
| `logger.total_ms` | `float` | Total execution time in milliseconds |
| `q.sql` | `str` | The SurrealQL query string |
| `q.duration_ms` | `float` | Execution time for this query |

The QueryLogger uses `contextvars` internally, which means it is async-safe. It only captures queries executed within the same async context (the `async with` block). Queries from other concurrent tasks are not captured.

In [ ]:
# Final cleanup: remove all test data
await TestUser.objects().delete_table()
print("Cleanup complete.")